In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from librosa import display
import librosa
import shutil
import csv

In [16]:
# Dataset Path
path = "../../Dataset/speech_dataset"

In [12]:
# Because there's no label of mini_command_speech
# This function return the metadata of each label

# New Filepath
pathFile = "setofdata"
newPath = path + "/" + pathFile

if not os.path.exists(newPath):
    
    # Make new folder if don't exist
    os.mkdir(newPath)
    
    # Open The File in The Write Mode
    f = open(newPath + "/record.csv", 'w')
    
    # Create The CSV Writer
    writer = csv.writer(f)
    counter = 0
    
    # Create Header
    dts = ["slice_file_name", "classID", "class"]
    writer.writerow(dts)
    
    for idx, folder in enumerate(os.listdir(path)):
        
        # Get the path of folder
        path_now = os.path.join(path, folder)
        
        # Iterate over index file
        for index, filename in enumerate(os.listdir(path_now)):
            file_old = os.path.join(path_now, filename)
            file_name = folder + "_" + str(index) + "_" + filename
            file_new = newPath + "/" + file_name
            
            if file_name.startswith(pathFile) == False:
                counter  = counter + 1
                dts = [file_name, idx, folder]
                writer.writerow(dts)
                shutil.copy2(file_old, file_new)

print(len(os.listdir(newPath)))
print(os.listdir(path))

KeyboardInterrupt: 

In [ ]:
os.path.isdir(newPath)

In [28]:
# Forming a Panda Dataframe From The Metadata file
data = pd.read_csv(newPath + "/record.csv")

# Head of The Dataframe
data.head()

# Pandas Data
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,slice_file_name,classID,class
0,stop_374_e9b61425_nohash_0.wav,0,stop
1,stop_533_caa4779f_nohash_1.wav,0,stop
2,stop_433_bbd0bbd0_nohash_3.wav,0,stop
3,stop_273_ea356919_nohash_1.wav,0,stop
4,stop_787_ee07dcb9_nohash_0.wav,0,stop


In [35]:
# Preprocessing using only MFCC
x_train = []
x_test  = []
y_train = []
y_test  = []

counter_pandas = 1
ratioTrain = int(len(data) * 0.8)
ratioTest  = len(data) - ratioTrain

for i in tqdm(range(len(data))):
    
    # Counter
    counter_pandas = counter_pandas + 1
    
    # Pandas get from data
    file = data.iloc[i]["slice_file_name"]
    label = data.iloc[i]["classID"]
    
    filename = newPath + "/" + file
    y, sr  = librosa.load(filename, 8000)
    mfccs = np.mean(librosa.feature.mfcc(y, sr, n_mfcc=40).T,axis=0)
    
    if counter_pandas <= ratioTrain : 
        x_train.append(mfccs)
        y_train.append(label)
    else:
        x_test.append(mfccs)
        y_test.append(label)    

100%|██████████| 846/846 [00:28<00:00, 29.81it/s]


In [36]:
# Converting the lists into Numpy Arrays
x_train = np.array(x_train)
x_test  = np.array(x_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

# Saving the data Numpy Arrays
np.savetxt("train_data.csv", x_train, delimiter=",")
np.savetxt("test_data.csv",x_test,delimiter=",")
np.savetxt("train_labels.csv",y_train,delimiter=",")
np.savetxt("test_labels.csv",y_test,delimiter=",")

In [37]:
x_test[0].shape

(40,)